In [1]:
import os
import numpy as np
import pandas as pd

from scipy.sparse import csr_matrix

from tqdm import tqdm

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import mean_squared_error, ndcg_score, recall_score
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

In [2]:
import torch
import torch.nn as nn
import torch.functional as F

from deepctr_torch.inputs import SparseFeat, VarLenSparseFeat, get_feature_names

In [3]:
from deepctr_torch.models import DIN
from deepctr_torch.models import AFM
from deepctr_torch.models import WDL
from deepctr_torch.models import xDeepFM
from deepctr_torch.models import DeepFM
from deepctr_torch.models import PNN
from deepctr_torch.models import DCN
from deepctr_torch.models import CCPM

In [4]:
import utils

In [5]:
device = 'cpu'
use_cuda = True
if use_cuda and torch.cuda.is_available():
    print('cuda ready...')
    device = 'cuda:0'

root = os.path.join(os.getcwd(), "DoubanBook")
rel_p = os.path.join(root, "user_book.dat")

cuda ready...


### User

In [6]:
group, group_i, group_m = utils.m2m_to_list(os.path.join(os.getcwd(), "DoubanBook", "user_group.dat"), user_cnt=13024)
location, location_i, location_m = utils.m2m_to_list(os.path.join(os.getcwd(), "DoubanBook", "user_location.dat"), user_cnt=13024)
user, user_i, user_m = utils.m2m_to_list(os.path.join(os.getcwd(), "DoubanBook", "user_user.dat"), user_cnt=13024)

### Book

In [7]:
year, year_i, year_m = utils.m2m_to_list(os.path.join(os.getcwd(), "DoubanBook", "book_year.dat"), user_cnt=22347)
publisher, publisher_i, publisher_m = utils.m2m_to_list(os.path.join(os.getcwd(), "DoubanBook", "book_publisher.dat"), user_cnt=22347)
author, author_i, author_m = utils.m2m_to_list(os.path.join(os.getcwd(), "DoubanBook", "book_author.dat"), user_cnt=22347)

### Interact

In [8]:
sparse_features = ["cols_0", "cols_1"] # user_id, movie_id
rating = "cols_2"

In [9]:
rel = utils.read_file(rel_p)
rel.head()

,cols_0,cols_1,cols_2
0,10855,938,4
1,10027,3,3
2,741,2426,5
3,453,1263,4
4,11665,7717,5


In [10]:
_group = group[rel.cols_0 -1]
_location = location[rel.cols_0 -1]
_user = user[rel.cols_0 -1]

_year = year[rel.cols_1 -1]
_publisher = publisher[rel.cols_1 -1]
_author = author[rel.cols_1 -1]

In [11]:
# 1.Label Encoding for sparse features,and process sequence features
for feat in sparse_features:
    lbe = LabelEncoder()
    rel[feat] = lbe.fit_transform(rel[feat])

In [12]:
fixlen_feature_columns = [SparseFeat(feat, rel[feat].nunique(), embedding_dim=4) for feat in sparse_features]

varlen_feature_columns = [
    VarLenSparseFeat(SparseFeat('group', vocabulary_size=group_i + 1, embedding_dim=4), maxlen=group_m, combiner='mean'),
    VarLenSparseFeat(SparseFeat('location', vocabulary_size=location_i + 1, embedding_dim=4), maxlen=location_m, combiner='mean'),
    VarLenSparseFeat(SparseFeat('user', vocabulary_size=13024 + 1, embedding_dim=4), maxlen=user_m, combiner='mean'),
    
    VarLenSparseFeat(SparseFeat('year', vocabulary_size=year_i + 1, embedding_dim=4), maxlen=year_m, combiner='mean'),
    VarLenSparseFeat(SparseFeat('publisher', vocabulary_size=publisher_i + 1, embedding_dim=4), maxlen=publisher_m, combiner='mean'),
    VarLenSparseFeat(SparseFeat('author', vocabulary_size=author_i + 1, embedding_dim=4), maxlen=author_m, combiner='mean')
]

In [13]:
linear_feature_columns = fixlen_feature_columns + varlen_feature_columns
dnn_feature_columns = fixlen_feature_columns + varlen_feature_columns

feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)

In [14]:
k = 5
kf = KFold(n_splits=5)

In [15]:
deepfm_mse = 0
ccpm_mse = 0
ipnn_mse = 0
opnn_mse = 0
wdl_mse = 0
dcn_mse = 0
xdeepfm_mse = 0
afm_mse = 0
din_mse = 0

In [ ]:
fold_cnt = 0

for train_index, test_index in kf.split(rel):
    
    fold_cnt += 1
    print("========================== {} Fold ==============================\n\n".format(fold_cnt))
    
    ### train
    train_input = {name: rel[name][train_index] for name in sparse_features}
    train_input["group"] = _group[train_index]
    train_input["location"] = _location[train_index]
    train_input["user"] = _user[train_index]
    train_input["year"] = _year[train_index]
    train_input["publisher"] = _publisher[train_index]
    train_input["author"] = _author[train_index]
    train_target = np.array(rel[rating][train_index])
    
    ### test
    test_input = {name: rel[name][test_index] for name in sparse_features}
    test_input["group"] = _group[test_index]
    test_input["location"] = _location[test_index]
    test_input["user"] = _user[test_index]
    test_input["year"] = _year[test_index]
    test_input["publisher"] = _publisher[test_index]
    test_input["author"] = _author[test_index]
    test_target = np.array(rel[rating][test_index])
    
    
    print("\n\n Training DeepFM \n")
    ### DeepFM
    deepfm = DeepFM(linear_feature_columns, dnn_feature_columns, task='regression', device=device)
    deepfm.compile("adam", "mse", metrics=['mse'], )
    deepfm_hist = deepfm.fit(train_input, train_target, batch_size=256, epochs=10, verbose=2, validation_split=0.1)
    pred_ans = deepfm.predict(test_input, batch_size=256)
    deepfm_mse += mean_squared_error(test_target, pred_ans)
    print("DeepFM MSE", round(mean_squared_error(test_target, pred_ans), 4))
    
    print("\n\n Training CCPM \n")
    ### CCPM
    ccpm = CCPM(linear_feature_columns, dnn_feature_columns, task='regression', device=device)
    ccpm.compile("adam", "mse", metrics=['mse'], )
    ccpm_hist = ccpm.fit(train_input, train_target, batch_size=256, epochs=10, verbose=2, validation_split=0.1)
    pred_ans = ccpm.predict(test_input, batch_size=256)
    ccpm_mse += mean_squared_error(test_target, pred_ans)
    print("CCPM MSE", round(mean_squared_error(test_target, pred_ans), 4))
    
    print("\n\n Training IPMM \n")
    ### IPNN
    ipnn = PNN(dnn_feature_columns, task='regression', device=device, use_inner=True, use_outter=False)
    ipnn.compile("adam", "mse", metrics=['mse'], )
    ipnn_hist = ipnn.fit(train_input, train_target, batch_size=256, epochs=10, verbose=2, validation_split=0.1)
    pred_ans = ipnn.predict(test_input, batch_size=256)
    ipnn_mse += mean_squared_error(test_target, pred_ans)
    print("IPNN MSE", round(mean_squared_error(test_target, pred_ans), 4))
    
    print("\n\n Training OPNN \n")
    ### OPNN
    opnn = PNN(dnn_feature_columns, task='regression', device=device, use_inner=False, use_outter=True)
    opnn.compile("adam", "mse", metrics=['mse'], )
    opnn_hist = opnn.fit(train_input, train_target, batch_size=256, epochs=10, verbose=2, validation_split=0.1)
    pred_ans = opnn.predict(test_input, batch_size=256)
    opnn_mse += mean_squared_error(test_target, pred_ans)
    print("OPNN MSE", round(mean_squared_error(test_target, pred_ans), 4))
    
    print("\n\n Training Wide&Deep \n")
    ### Wide & Deep
    wdl = WDL(linear_feature_columns, dnn_feature_columns, task='regression', device=device)
    wdl.compile("adam", "mse", metrics=['mse'], )
    wdl_hist = wdl.fit(train_input, train_target, batch_size=256, epochs=10, verbose=2, validation_split=0.1)
    pred_ans = wdl.predict(test_input, batch_size=256)
    wdl_mse += mean_squared_error(test_target, pred_ans)
    print("WDL MSE", round(mean_squared_error(test_target, pred_ans), 4))
    
    print("\n\n Training DCN \n")
    ### DCN
    dcn = DCN(linear_feature_columns, dnn_feature_columns, task='regression', device=device)
    dcn.compile("adam", "mse", metrics=['mse'], )
    dcn_hist = dcn.fit(train_input, train_target, batch_size=256, epochs=10, verbose=2, validation_split=0.1)
    pred_ans = dcn.predict(test_input, batch_size=256)
    dcn_mse += mean_squared_error(test_target, pred_ans)
    print("DCN MSE", round(mean_squared_error(test_target, pred_ans), 4))
    
    
    print("\n\n Training xDeepFM \n")
    ### xDeepFM
    xdeepfm = xDeepFM(linear_feature_columns, dnn_feature_columns, task='regression', device=device)
    xdeepfm.compile("adam", "mse", metrics=['mse'], )
    xdeepfm_hist = xdeepfm.fit(train_input, train_target, batch_size=256, epochs=10, verbose=2, validation_split=0.1)
    pred_ans = xdeepfm.predict(test_input, batch_size=256)
    xdeepfm_mse += mean_squared_error(test_target, pred_ans)
    print("xDeepFM MSE", round(mean_squared_error(test_target, pred_ans), 4))
    
    
    print("\n\n Training AFM \n")
    ### AFM
    afm = AFM(linear_feature_columns, dnn_feature_columns, task='regression', device=device)
    afm.compile("adam", "mse", metrics=['mse'], )
    afm_hist = afm.fit(train_input, train_target, batch_size=256, epochs=10, verbose=2, validation_split=0.1)
    pred_ans = afm.predict(test_input, batch_size=256)
    afm_mse += mean_squared_error(test_target, pred_ans)
    print("AFM MSE", round(mean_squared_error(test_target, pred_ans), 4))
    
    """
    print("\n Training DIN \n")
    ### DIN
    din = DIN(dnn_feature_columns, [], task='regression', device=device)
    din.compile("adam", "mse", metrics=['mse'], )
    din_hist = din.fit(train_input, train_target, batch_size=256, epochs=10, verbose=2, validation_split=0.1)
    pred_ans = din.predict(test_input, batch_size=256)
    din_mse += mean_squared_error(test_target, pred_ans)
    print("DIN MSE", round(mean_squared_error(test_target, pred_ans), 4))
    """

========================== 1 Fold ==============================




 Training DeepFM 

cuda:0
Train on 570284 samples, validate on 63365 samples, 2228 steps per epoch
Epoch 1/10
299s - loss:  0.7355 - mse:  0.7354 - val_mse:  0.4965
Epoch 2/10
306s - loss:  0.4834 - mse:  0.4834 - val_mse:  0.4935
Epoch 3/10
306s - loss:  0.4734 - mse:  0.4733 - val_mse:  0.4899
Epoch 4/10
294s - loss:  0.4672 - mse:  0.4672 - val_mse:  0.4881
Epoch 5/10
334s - loss:  0.4614 - mse:  0.4613 - val_mse:  0.4903
Epoch 6/10
350s - loss:  0.4553 - mse:  0.4552 - val_mse:  0.4886
Epoch 7/10
350s - loss:  0.4470 - mse:  0.4469 - val_mse:  0.4873
Epoch 8/10
347s - loss:  0.4363 - mse:  0.4361 - val_mse:  0.4893
Epoch 9/10
354s - loss:  0.4219 - mse:  0.4216 - val_mse:  0.4982
Epoch 10/10
352s - loss:  0.4049 - mse:  0.4045 - val_mse:  0.5070
DeepFM MSE 0.5089


 Training CCPM 

cuda:0
Train on 570284 samples, validate on 63365 samples, 2228 steps per epoch
Epoch 1/10
376s - loss:  0.6663 - mse:  0.6663 - val_m

In [ ]:
deepfm_mse /5

In [ ]:
ccpm_mse /5

In [ ]:
ipnn_mse /5

In [ ]:
opnn_mse /5

In [ ]:
wdl_mse /5

In [ ]:
dcn_mse /5

In [ ]:
xdeepfm_mse /5

In [ ]:
afm_mse /5

In [ ]:
din_mse /5

# Test single model

In [38]:
train_p = os.path.join(root, "train.dat")

In [39]:
group, group_i, group_m = utils.m2m_to_list(os.path.join(os.getcwd(), "DoubanBook", "user_group.dat"), user_cnt=13024)
location, location_i, location_m = utils.m2m_to_list(os.path.join(os.getcwd(), "DoubanBook", "user_location.dat"), user_cnt=13024)
user, user_i, user_m = utils.m2m_to_list(os.path.join(os.getcwd(), "DoubanBook", "user_user.dat"), user_cnt=13024)

In [40]:
year, year_i, year_m = utils.m2m_to_list(os.path.join(os.getcwd(), "DoubanBook", "book_year.dat"), user_cnt=22347)
publisher, publisher_i, publisher_m = utils.m2m_to_list(os.path.join(os.getcwd(), "DoubanBook", "book_publisher.dat"), user_cnt=22347)
author, author_i, author_m = utils.m2m_to_list(os.path.join(os.getcwd(), "DoubanBook", "book_author.dat"), user_cnt=22347)

In [41]:
sparse_features = ["cols_0", "cols_1"] # user_id, movie_id
rating = "cols_2"

In [42]:
train = utils.read_file(train_p)
train.head()

,cols_0,cols_1,cols_2
0,9608,791,3
1,11607,2664,4
2,3630,712,4
3,12070,5046,5
4,3967,202,5


In [43]:
_group = group[rel.cols_0 -1]
_location = location[rel.cols_0 -1]
_user = user[rel.cols_0 -1]

_year = year[rel.cols_1 -1]
_publisher = publisher[rel.cols_1 -1]
_author = author[rel.cols_1 -1]

In [44]:
# 1.Label Encoding for sparse features,and process sequence features
for feat in sparse_features:
    lbe = LabelEncoder()
    train[feat] = lbe.fit_transform(train[feat])

In [45]:
fixlen_feature_columns = [SparseFeat(feat, train[feat].nunique(), embedding_dim=4) for feat in sparse_features]

varlen_feature_columns = [
    VarLenSparseFeat(SparseFeat('group', vocabulary_size=group_i + 1, embedding_dim=4), maxlen=group_m, combiner='mean'),
    VarLenSparseFeat(SparseFeat('location', vocabulary_size=location_i + 1, embedding_dim=4), maxlen=location_m, combiner='mean'),
    VarLenSparseFeat(SparseFeat('user', vocabulary_size=13024 + 1, embedding_dim=4), maxlen=user_m, combiner='mean'),
    
    VarLenSparseFeat(SparseFeat('year', vocabulary_size=year_i + 1, embedding_dim=4), maxlen=year_m, combiner='mean'),
    VarLenSparseFeat(SparseFeat('publisher', vocabulary_size=publisher_i + 1, embedding_dim=4), maxlen=publisher_m, combiner='mean'),
    VarLenSparseFeat(SparseFeat('author', vocabulary_size=author_i + 1, embedding_dim=4), maxlen=author_m, combiner='mean')
]

In [46]:
linear_feature_columns = fixlen_feature_columns + varlen_feature_columns
dnn_feature_columns = fixlen_feature_columns + varlen_feature_columns

feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)

In [47]:
# 3.generate input data for model
model_input = {name: train[name] for name in sparse_features}  #
model_input["group"] = _group
model_input["location"] = _location
model_input["user"] = _user
model_input["year"] = _year
model_input["publisher"] = _publisher
model_input["author"] = _author

### DeepFM

In [15]:
from deepctr_torch.models import DeepFM

In [16]:
deepfm = DeepFM(linear_feature_columns, dnn_feature_columns, task='regression', device=device)

deepfm.compile("adam", "mse", metrics=['mse'], )
history = deepfm.fit(model_input, train_df[rating].values, batch_size=256, epochs=10, verbose=2, validation_split=0.2)

cuda:0
Train on 456227 samples, validate on 114057 samples, 1783 steps per epoch
Epoch 1/10
228s - loss:  0.7947 - mse:  0.7945 - val_mse:  0.5051
Epoch 2/10
230s - loss:  0.4835 - mse:  0.4835 - val_mse:  0.5019
Epoch 3/10
228s - loss:  0.4719 - mse:  0.4718 - val_mse:  0.4993
Epoch 4/10
226s - loss:  0.4649 - mse:  0.4649 - val_mse:  0.5034
Epoch 5/10
232s - loss:  0.4600 - mse:  0.4599 - val_mse:  0.4983
Epoch 6/10
226s - loss:  0.4546 - mse:  0.4545 - val_mse:  0.5073
Epoch 7/10
228s - loss:  0.4494 - mse:  0.4492 - val_mse:  0.5018
Epoch 8/10
229s - loss:  0.4428 - mse:  0.4426 - val_mse:  0.5088
Epoch 9/10
228s - loss:  0.4335 - mse:  0.4333 - val_mse:  0.4998
Epoch 10/10
229s - loss:  0.4216 - mse:  0.4213 - val_mse:  0.5087


### CCPM

In [17]:
from deepctr_torch.models import CCPM

In [18]:
ccpm = CCPM(linear_feature_columns, dnn_feature_columns, task='regression', device=device)

ccpm.compile("adam", "mse", metrics=['mse'], )
history = ccpm.fit(model_input, train_df[rating].values, batch_size=256, epochs=10, verbose=2, validation_split=0.2)

cuda:0
Train on 456227 samples, validate on 114057 samples, 1783 steps per epoch
Epoch 1/10
228s - loss:  0.7249 - mse:  0.7248 - val_mse:  0.5026
Epoch 2/10
229s - loss:  0.4766 - mse:  0.4765 - val_mse:  0.4946
Epoch 3/10
224s - loss:  0.4588 - mse:  0.4588 - val_mse:  0.4915
Epoch 4/10
231s - loss:  0.4455 - mse:  0.4456 - val_mse:  0.4947
Epoch 5/10
237s - loss:  0.4302 - mse:  0.4300 - val_mse:  0.4993
Epoch 6/10
232s - loss:  0.4124 - mse:  0.4122 - val_mse:  0.5155
Epoch 7/10
207s - loss:  0.3938 - mse:  0.3936 - val_mse:  0.5194
Epoch 8/10
210s - loss:  0.3771 - mse:  0.3770 - val_mse:  0.5336
Epoch 9/10
214s - loss:  0.3616 - mse:  0.3613 - val_mse:  0.5405
Epoch 10/10
213s - loss:  0.3485 - mse:  0.3483 - val_mse:  0.5592


### PNN

In [19]:
from deepctr_torch.models import PNN

#### IPNN

In [20]:
ipnn = PNN(dnn_feature_columns, task='regression', device=device, use_inner=True, use_outter=False)

In [21]:
ipnn.compile("adam", "mse", metrics=['mse'], )
history = ipnn.fit(model_input, train_df[rating].values, batch_size=256, epochs=10, verbose=2, validation_split=0.2)

cuda:0
Train on 456227 samples, validate on 114057 samples, 1783 steps per epoch
Epoch 1/10
125s - loss:  0.9133 - mse:  0.9131 - val_mse:  0.5042
Epoch 2/10
125s - loss:  0.4784 - mse:  0.4784 - val_mse:  0.4927
Epoch 3/10
123s - loss:  0.4554 - mse:  0.4554 - val_mse:  0.4902
Epoch 4/10
124s - loss:  0.4353 - mse:  0.4353 - val_mse:  0.4972
Epoch 5/10
128s - loss:  0.4149 - mse:  0.4150 - val_mse:  0.5067
Epoch 6/10
128s - loss:  0.3920 - mse:  0.3918 - val_mse:  0.5238
Epoch 7/10
128s - loss:  0.3712 - mse:  0.3711 - val_mse:  0.5402
Epoch 8/10
136s - loss:  0.3534 - mse:  0.3532 - val_mse:  0.5562
Epoch 9/10
141s - loss:  0.3391 - mse:  0.3390 - val_mse:  0.5750
Epoch 10/10
136s - loss:  0.3282 - mse:  0.3280 - val_mse:  0.5825


#### OPNN

In [22]:
opnn = PNN(dnn_feature_columns, task='regression', device=device, use_inner=False, use_outter=True)

In [23]:
opnn.compile("adam", "mse", metrics=['mse'], )
history = opnn.fit(model_input, train_df[rating].values, batch_size=256, epochs=10, verbose=2, validation_split=0.2)

cuda:0
Train on 456227 samples, validate on 114057 samples, 1783 steps per epoch
Epoch 1/10
142s - loss:  0.8448 - mse:  0.8446 - val_mse:  0.5039
Epoch 2/10
143s - loss:  0.4802 - mse:  0.4802 - val_mse:  0.4969
Epoch 3/10
141s - loss:  0.4659 - mse:  0.4658 - val_mse:  0.4955
Epoch 4/10
149s - loss:  0.4523 - mse:  0.4523 - val_mse:  0.4966
Epoch 5/10
145s - loss:  0.4381 - mse:  0.4381 - val_mse:  0.4894
Epoch 6/10
148s - loss:  0.4243 - mse:  0.4242 - val_mse:  0.4984
Epoch 7/10
149s - loss:  0.4031 - mse:  0.4031 - val_mse:  0.5068
Epoch 8/10
148s - loss:  0.3811 - mse:  0.3809 - val_mse:  0.5243
Epoch 9/10
149s - loss:  0.3619 - mse:  0.3618 - val_mse:  0.5375
Epoch 10/10
150s - loss:  0.3465 - mse:  0.3463 - val_mse:  0.5589


#### PIN

### Wide & Deep

In [24]:
from deepctr_torch.models import WDL

In [25]:
wdl = WDL(linear_feature_columns, dnn_feature_columns, task='regression', device=device)

In [26]:
wdl.compile("adam", "mse", metrics=['mse'], )
history = wdl.fit(model_input, train_df[rating].values, batch_size=256, epochs=10, verbose=2, validation_split=0.2)

cuda:0
Train on 456227 samples, validate on 114057 samples, 1783 steps per epoch
Epoch 1/10
246s - loss:  0.7961 - mse:  0.7960 - val_mse:  0.5040
Epoch 2/10
240s - loss:  0.4823 - mse:  0.4823 - val_mse:  0.5012
Epoch 3/10
246s - loss:  0.4711 - mse:  0.4710 - val_mse:  0.4985
Epoch 4/10
250s - loss:  0.4652 - mse:  0.4651 - val_mse:  0.5034
Epoch 5/10
244s - loss:  0.4620 - mse:  0.4618 - val_mse:  0.4979
Epoch 6/10
246s - loss:  0.4590 - mse:  0.4589 - val_mse:  0.5057
Epoch 7/10
227s - loss:  0.4565 - mse:  0.4564 - val_mse:  0.5003
Epoch 8/10
216s - loss:  0.4539 - mse:  0.4538 - val_mse:  0.5046
Epoch 9/10
224s - loss:  0.4500 - mse:  0.4499 - val_mse:  0.4941
Epoch 10/10
246s - loss:  0.4450 - mse:  0.4449 - val_mse:  0.4990


### Deep Cross

In [27]:
from deepctr_torch.models import DCN

In [28]:
dcn = DCN(linear_feature_columns, dnn_feature_columns, task='regression', device=device)

In [29]:
dcn.compile("adam", "mse", metrics=['mse'], )
history = dcn.fit(model_input, train_df[rating].values, batch_size=256, epochs=10, verbose=2, validation_split=0.2)

cuda:0
Train on 456227 samples, validate on 114057 samples, 1783 steps per epoch
Epoch 1/10
258s - loss:  0.8089 - mse:  0.8087 - val_mse:  0.5125
Epoch 2/10
261s - loss:  0.4816 - mse:  0.4817 - val_mse:  0.4968
Epoch 3/10
261s - loss:  0.4698 - mse:  0.4698 - val_mse:  0.4977
Epoch 4/10
257s - loss:  0.4629 - mse:  0.4628 - val_mse:  0.4945
Epoch 5/10
255s - loss:  0.4569 - mse:  0.4567 - val_mse:  0.4912
Epoch 6/10
258s - loss:  0.4486 - mse:  0.4486 - val_mse:  0.4922
Epoch 7/10
252s - loss:  0.4416 - mse:  0.4415 - val_mse:  0.4916
Epoch 8/10
251s - loss:  0.4345 - mse:  0.4343 - val_mse:  0.4934
Epoch 9/10
256s - loss:  0.4284 - mse:  0.4283 - val_mse:  0.4968
Epoch 10/10
263s - loss:  0.4220 - mse:  0.4217 - val_mse:  0.5029


### xDeepFM

In [30]:
from deepctr_torch.models import xDeepFM

In [31]:
xdeepfm = xDeepFM(linear_feature_columns, dnn_feature_columns, task='regression', device=device)

In [32]:
xdeepfm.compile("adam", "mse", metrics=['mse'], )
history = xdeepfm.fit(model_input, train_df[rating].values, batch_size=256, epochs=10, verbose=2, validation_split=0.2)

cuda:0
Train on 456227 samples, validate on 114057 samples, 1783 steps per epoch
Epoch 1/10
267s - loss:  0.6677 - mse:  0.6676 - val_mse:  0.5041
Epoch 2/10
262s - loss:  0.4801 - mse:  0.4800 - val_mse:  0.5075
Epoch 3/10
260s - loss:  0.4634 - mse:  0.4634 - val_mse:  0.4897
Epoch 4/10
259s - loss:  0.4514 - mse:  0.4513 - val_mse:  0.4937
Epoch 5/10
266s - loss:  0.4428 - mse:  0.4428 - val_mse:  0.4951
Epoch 6/10
261s - loss:  0.4348 - mse:  0.4347 - val_mse:  0.4953
Epoch 7/10
262s - loss:  0.4266 - mse:  0.4264 - val_mse:  0.4972
Epoch 8/10
253s - loss:  0.4124 - mse:  0.4123 - val_mse:  0.5075
Epoch 9/10
265s - loss:  0.3919 - mse:  0.3917 - val_mse:  0.5153
Epoch 10/10
268s - loss:  0.3690 - mse:  0.3688 - val_mse:  0.5356


### Attentional Factorization Machine

In [33]:
from deepctr_torch.models import AFM

In [34]:
afm = AFM(linear_feature_columns, dnn_feature_columns, task='regression', device=device)

In [35]:
afm.compile("adam", "mse", metrics=['mse'], )
history = afm.fit(model_input, train_df[rating].values, batch_size=256, epochs=10, verbose=2, validation_split=0.2)

cuda:0
Train on 456227 samples, validate on 114057 samples, 1783 steps per epoch
Epoch 1/10
255s - loss:  2.5075 - mse:  2.5064 - val_mse:  0.6100
Epoch 2/10
245s - loss:  0.5585 - mse:  0.5580 - val_mse:  0.5407
Epoch 3/10
250s - loss:  0.5036 - mse:  0.5033 - val_mse:  0.5176
Epoch 4/10
248s - loss:  0.4790 - mse:  0.4785 - val_mse:  0.5092
Epoch 5/10
255s - loss:  0.4663 - mse:  0.4658 - val_mse:  0.5048
Epoch 6/10
251s - loss:  0.4587 - mse:  0.4581 - val_mse:  0.5022
Epoch 7/10
253s - loss:  0.4536 - mse:  0.4529 - val_mse:  0.5014
Epoch 8/10
251s - loss:  0.4497 - mse:  0.4490 - val_mse:  0.5012
Epoch 9/10
256s - loss:  0.4468 - mse:  0.4461 - val_mse:  0.5005
Epoch 10/10
254s - loss:  0.4443 - mse:  0.4436 - val_mse:  0.5011


### Neural Factorization Machine

In [36]:
from deepctr_torch.models import NFM

In [37]:
nfm = NFM(linear_feature_columns, dnn_feature_columns, task='regression', device=device)

In [38]:
nfm.compile("adam", "mse", metrics=['mse'], )
history = nfm.fit(model_input, train_df[rating].values, batch_size=256, epochs=10, verbose=2, validation_split=0.2)

cuda:0
Train on 456227 samples, validate on 114057 samples, 1783 steps per epoch
Epoch 1/10
247s - loss:  0.7533 - mse:  0.7532 - val_mse:  0.5090
Epoch 2/10
234s - loss:  0.4786 - mse:  0.4785 - val_mse:  0.4960
Epoch 3/10
246s - loss:  0.4566 - mse:  0.4565 - val_mse:  0.4959
Epoch 4/10
246s - loss:  0.4432 - mse:  0.4432 - val_mse:  0.4988
Epoch 5/10
242s - loss:  0.4338 - mse:  0.4337 - val_mse:  0.5024
Epoch 6/10
248s - loss:  0.4259 - mse:  0.4258 - val_mse:  0.5026
Epoch 7/10
238s - loss:  0.4191 - mse:  0.4189 - val_mse:  0.5034
Epoch 8/10
248s - loss:  0.4125 - mse:  0.4123 - val_mse:  0.5059
Epoch 9/10
236s - loss:  0.4040 - mse:  0.4037 - val_mse:  0.5117
Epoch 10/10
230s - loss:  0.3946 - mse:  0.3943 - val_mse:  0.5179


### Deep Interest Network

In [48]:
from deepctr_torch.models import DIN

In [51]:
behavior_feature_list = np.array(["cols_0", "cols_1"])

In [52]:
din = DIN(dnn_feature_columns, behavior_feature_list, task='regression', device=device)

In [53]:
din.compile("adam", "mse", metrics=['mse'], )
history = din.fit(model_input, train[rating].values, batch_size=256, epochs=10, verbose=2, validation_split=0.2)

cuda:0
Train on 456227 samples, validate on 114057 samples, 1783 steps per epoch


AttributeError: 'VarLenSparseFeat' object has no attribute 'use_hash'